#Troisième partie : modélisation

In [2]:
# on réimporte les librairies
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd # on s'en sert juste pour un affichage un peu plus propre de certains résultats (coefficients des régressions et erreur par pays à la fin)

In [3]:
# on réimporte le dataframe sauvegardé enfin de partie 2 au format "parquet"
df = spark.read.load("/FileStore/parquet/airbnb_part_2")
display(df.limit(50))

country,description_len,nb_desc,apartment,house,hostel,cancel_strict,cancel_moderate,is_location_exact,host_identity_verified,instant_bookable,host_is_superhost,host_response_rate,bathrooms,accommodates,bedrooms,price_per_person,extra_people,minimum_nights,availability_365,number_of_reviews,review_scores_rating,reviews_per_month,amenities_nb,tv,cable_tv,free_parking,pool,gym,air_conditioning,elevator,detectors,appliances,pets,disable,kids,smoking,smart_clothes,fireplace,easy_check_in,breakfast,tub,events,privacy,is_depot,experiences,room_type_dummies,bed_type_dummies,market_dummies,country_dummies,host_response_time_dummies
France,37,3,1,0,0,1,0,1,1,0,0,90,1.0,2.0,1,47,0.0,2,244.0,31,92.0,2.02,12,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(0), List(1.0))","List(0, 15, List(2), List(1.0))","List(0, 4, List(0), List(1.0))"
United Kingdom,40,2,0,1,0,0,0,1,0,0,0,100,1.0,1.0,1,15,0.0,1,0.0,1,100.0,0.07,10,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,"List(0, 2, List(1), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(1), List(1.0))","List(0, 15, List(1), List(1.0))","List(0, 4, List(2), List(1.0))"
United States,127,3,1,0,0,1,0,1,0,1,0,100,1.0,3.0,1,128,25.0,6,153.0,30,83.0,1.85,21,1,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(2), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(1), List(1.0))"
United States,168,6,0,1,0,1,0,1,1,0,0,100,1.0,4.0,1,51,0.0,3,330.0,17,95.0,0.38,24,1,1,1,0,0,1,0,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(3), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(1), List(1.0))"
France,127,8,1,0,0,0,0,1,0,1,0,100,1.0,2.0,1,24,10.0,1,56.0,69,74.0,4.6,7,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,"List(0, 2, List(1), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(0), List(1.0))","List(0, 15, List(2), List(1.0))","List(0, 4, List(0), List(1.0))"
United States,175,7,1,0,0,0,1,1,1,0,0,100,1.0,4.0,1,27,20.0,2,97.0,214,97.0,5.68,19,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(33), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(0), List(1.0))"
Canada,192,8,1,0,0,0,0,0,0,0,0,100,1.0,3.0,1,33,0.0,1,0.0,4,70.0,0.23,22,1,0,0,0,1,1,1,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(12), List(1.0))","List(0, 15, List(6), List(1.0))","List(0, 4, List(2), List(1.0))"
United States,130,9,1,0,0,1,0,1,1,0,0,100,1.0,2.0,1,67,20.0,1,0.0,2,100.0,0.14,18,0,0,1,0,0,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,"List(0, 2, List(1), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(3), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(2), List(1.0))"
United States,178,10,0,1,0,0,1,1,0,0,0,90,1.0,3.0,1,46,30.0,2,345.0,36,99.0,2.4,20,1,1,1,0,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,"List(0, 2, List(1), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(24), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(1), List(1.0))"
United States,181,8,1,0,0,1,0,1,1,1,0,100,1.0,3.0,0,70,25.0,3,71.0,49,97.0,2.97,26,1,1,0,0,1,1,1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,"List(0, 2, List(0), List(1.0))","List(0, 4, List(0), List(1.0))","List(0, 51, List(2), List(1.0))","List(0, 15, List(0), List(1.0))","List(0, 4, List(0), List(1.0))"


On vérifie qu'on a bien que des variables numériques

In [5]:
df.printSchema()

root
-- country: string (nullable = true)
-- description_len: integer (nullable = true)
-- nb_desc: integer (nullable = true)
-- apartment: integer (nullable = true)
-- house: integer (nullable = true)
-- hostel: integer (nullable = true)
-- cancel_strict: integer (nullable = true)
-- cancel_moderate: integer (nullable = true)
-- is_location_exact: integer (nullable = true)
-- host_identity_verified: integer (nullable = true)
-- instant_bookable: integer (nullable = true)
-- host_is_superhost: integer (nullable = true)
-- host_response_rate: integer (nullable = true)
-- bathrooms: double (nullable = true)
-- accommodates: double (nullable = true)
-- bedrooms: long (nullable = true)
-- price_per_person: integer (nullable = true)
-- extra_people: double (nullable = true)
-- minimum_nights: integer (nullable = true)
-- availability_365: double (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- review_scores_rating: double (nullable = true)
-- reviews_per_month: string (nullable = true)
-- amenities_nb: integer (nullable = true)
-- tv: integer (nullable = true)
-- cable_tv: integer (nullable = true)
-- free_parking: integer (nullable = true)
-- pool: integer (nullable = true)
-- gym: integer (nullable = true)
-- air_conditioning: integer (nullable = true)
-- elevator: integer (nullable = true)
-- detectors: integer (nullable = true)
-- appliances: integer (nullable = true)
-- pets: integer (nullable = true)
-- disable: integer (nullable = true)
-- kids: integer (nullable = true)
-- smoking: integer (nullable = true)
-- smart_clothes: integer (nullable = true)
-- fireplace: integer (nullable = true)
-- easy_check_in: integer (nullable = true)
-- breakfast: integer (nullable = true)
-- tub: integer (nullable = true)
-- events: integer (nullable = true)
-- privacy: integer (nullable = true)
-- is_depot: integer (nullable = true)
-- experiences: integer (nullable = true)
-- room_type_dummies: vector (nullable = true)
-- bed_type_dummies: vector (nullable = true)
-- market_dummies: vector (nullable = true)
-- country_dummies: vector (nullable = true)
-- host_response_time_dummies: vector (nullable = true)

In [6]:
# il reste 2 variables caractère : country dont on a pas besoin pour le moment et reviews_per_month alors on change ça vite fait
df_num = df.drop("country").withColumn("reviews_per_month", df["reviews_per_month"].cast(FloatType()))

Dans une première partie régression, on ne va pas conserver à la fois les villes et les pays. On va donc s'en tenir aux pays pour commencer histoire de limiter le un peu le nombre d'indicatrices.

In [8]:
df_num = df_num.drop("market_dummies")

On vérifie s'il reste des valeurs nulles. Si oui, on pourra les remplacer par 0 même si c'est un traitement bif bof à éviter...

In [10]:
#nulls = df_num.select([count(when(col(colname).isNull(), colname)).alias(colname) for colname in df_num.columns]).toPandas()
#nulls
# output : génial nobody.isNull

## Modèle pour expliquer les notes  
L'idée est de voir quelle est l'influence des caractéristiques du logement sur les notes reçues par l'hôte.

In [12]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("review_scores_rating").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [13]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='review_scores_rating', maxIter=100)
lr_model = lr.fit(df_reg)

Afin de récupérer les coefficients, il y a un petit boulot à faire car les dummy variables construites avec OneHotEncoderEstimator n'ont pas de noms de variable associés. Pour rappel lorsqu'on utilise le one hot encoding de pyspark, on a une variable, par exemple `country_dummies` qui est un sparse vector contenant toutes les dummies

Par conséquent, on a dans `name_num` une 40aine de variable alors qu'on a dans `lr_model.coefficients` une 60aine de valeurs...ce qui est normal puisque certaines variables (les dummies) en contiennent en fait plusieurs. Il faut donc pouvoir faire correspondre chaque coefficient à la bonne variable (et notamment pour ces dummies).

In [15]:
### 1ère étape : construction de la liste des noms de variable en incluant les dummies
# pour ce la on peut passer par les métadonnées du modèle de régression

# application du modèle linéaire sur le df pour obtenir les prédictions
pred = lr_model.transform(df_reg)

# récupération des métadonnées
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]

# récupération des index et des noms des variables du modèle
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']

# création de la liste des noms de variables du modèle
variables = [item['name'] for item in features_name_index]

#### 2ème étape : les coefficients et les variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
country_dummies_Hong Kong,-5.452420
host_response_time_dummies_not available,-3.456667
host_response_time_dummies_within a few hours,-2.727357
host_response_time_dummies_within an hour,-2.701081
host_response_time_dummies_within a day,-2.072026
country_dummies_Denmark,-1.694869
instant_bookable,-1.645944
country_dummies_Ireland,-1.399586
smoking,-1.240503
cancel_strict,-1.092345


In [16]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.08821951273403805
R2 ajusté: 0.08804588675225433

Le pouvoir prédictif du modèle est assez faible (R2 de 8,8 %) mais on constate quelques résultats intéressants : le fait de louer un logement entier augmente la note reçue par rapport aux autres logements de 2,1. On constate également que plus on a de reviews, plus la note augmente. Le prix a par contre un effet très faible sur la note reçue. Le fait qu'il y ait une télé, des détecteurs de fumée ou un petit déjeuner inclus augmente aussi sensiblement la note en moyenne, alors que le fait de pouvoir fumer la diminue. En revanche, on voit que certaines caractéristiques de l'hôte notamment sur le temps de réponse ont des effets très négatifs sur la note ce qui est assez contre-intuitif...cela semble rejoindre l'idée que le modèle de prédiction de la note n'est pas vraiment adapté puisque la note dépend plus du séjour en lui-même (sur lequel nous n'avons malheureusement pas d'autre information) que de la partie préliminaire qui consiste à sélectionner le logement. Concentrons-nous donc sur le prix et le nombre d'avis mensuel (comme proxy du nombre de visiteurs).

## Modèle pour expliquer le prix

In [19]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("price_per_person").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [20]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='price_per_person', maxIter=100)
lr_model = lr.fit(df_reg)

In [21]:
# les coefficients

# liste des noms de variables du modèle
pred = lr_model.transform(df_reg)
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']
variables = [item['name'] for item in features_name_index]

# coefficients et variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
country_dummies_Greece,-36.787665
country_dummies_Austria,-26.530276
country_dummies_Belgium,-25.066514
country_dummies_Spain,-24.625781
country_dummies_Germany,-22.319655
country_dummies_France,-21.011678
country_dummies_Italy,-19.962519
country_dummies_United Kingdom,-15.888309
country_dummies_Ireland,-9.827038
free_parking,-8.637254


In [22]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.725781727291789
R2 ajusté: 0.7257295092223228

Le pouvoir prédictif du modèle est intéressant (R2 de 72,6 %). On en fera donc un modèle prédictif plus loin dans le notebook. On constate que le prix par personne est le plus important pour les hotels et les appartements, et pas pour les maisons, sans doute parce qu'elles peuvent accueillir plus de monde. On constate un effet très faible mais positif des notes reçues par l'hôte avec le prix, ce qui peut s'interpréter ainsi : un hôte avec des bonnes notes peut se permettre de tarifer plus cher que celui qui a de mauvaises notes et est contraint de convaincre le client de réserver en baissant son prix. Le fait d'organiser des évènement lors du séjour augmente le prix (de 3,2 $) et le fait qu'il y ait un dépôt de garantie aussi (2,6 $). L'organisation d'expériences par l'hôte augmente toutes choses égales par ailleurs le prix de 0,45 $.

## Modèle pour expliquer le nombre de notes reçues
Le nombre de retours est un indicateur sur l'attractivité d'un logement puisqu'il permet d'avoir une estimation du nombre de demandes reçues par le propriétaire.

In [25]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("reviews_per_month").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [26]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='reviews_per_month', maxIter=100, regParam=0.0, elasticNetParam=0.0)
lr_model = lr.fit(df_reg)

In [27]:
# les coefficients

# liste des noms de variables du modèle
pred = lr_model.transform(df_reg)
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']
variables = [item['name'] for item in features_name_index]

# coefficients et variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
experiences,-0.606943
country_dummies_Italy,-0.574401
country_dummies_Greece,-0.460107
country_dummies_Spain,-0.458656
country_dummies_Germany,-0.449761
country_dummies_France,-0.429329
country_dummies_Netherlands,-0.377495
country_dummies_Austria,-0.364295
host_response_time_dummies_not available,-0.346880
country_dummies_United Kingdom,-0.296680


In [28]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.4485083146300639
R2 ajusté: 0.4484032967154944

Le pouvoir prédictif du modèle est intéressant (R2 de 44,8 %). On constate que les variables ayant le plus d'effet sur le nombre de notes reçues par mois sont la variable indicatrice indiquant si le logement peut être réservé instantanément (`instant_bookable`) et celle indiquant si l'hôte répond généralement en moins d'1h (`host_response_time_dummies_within an hour`). Ces caractéristiques augmentent le nombre de reviews par mois de respectivement 0,47 et 0,68 en moyenne et donc le nombre de réservations. On constate par contre une absence d'effet du fait d'être un "super-hôte" airbnb.

## Modèle prédictif du prix  
On peut imaginer un modèle visant à aider les propriétaires à ajuster le prix de leur nuitée en fonction des caractéristiques de leur logement.

On sort quelques stats sur les prix pour voir sur quels ordres de grandeur on est :

In [32]:
# On compare aux statistiques sur le prix : 
df_reg.select("price_per_person").describe().show()

+-------+------------------+
summary| price_per_person|
+-------+------------------+
 count| 362417|
 mean|61.611116476324234|
 stddev| 66.15269739544337|
 min| 0|
 max| 1201|
+-------+------------------+

On essaye un première modèle de régression linéaire classique :

In [34]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("price_per_person").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

# On splite notre base
splits = df_reg.randomSplit([0.7, 0.3])
train_price = splits[0]
test_price = splits[1]

lr = LinearRegression(featuresCol = 'var_exp', labelCol='price_per_person', maxIter=100)
lr_train_test = lr.fit(train_price)

# on applique la prédiction
lr_predictions = lr_train_test.transform(test_price)

# on sort MAE et rmse pour juger de la qualité de la prédiction
test_result = lr_train_test.evaluate(test_price)
# RMSE & MAE
print(test_result.rootMeanSquaredError, test_result.meanAbsoluteError)

34.45003568757711 19.550886713756242

L'erreur moyenne de prédiction est autour de 19,5$.  
On peut faire varier `elasticNetParam`, qui permet de pénaliser le modèle, et voir les effets sur la qualité de la prédiction. On retombe en effet sur un modèle de Lasso quand il est égal à 1 et sur un modèle de Ridge quand il est égal à 0 (voir [la documentation](https://spark.apache.org/docs/1.5.2/ml-linear-methods.html)).

In [36]:
from pyspark.ml.feature import MinMaxScaler
# On fait un scaler avant d'appliquer la régression pénalisée
scaler = MinMaxScaler(inputCol="var_exp", outputCol="var_exp_scale")
scaler_model = scaler.fit(train_price)
train_price_scale = scaler_model.transform(train_price)
test_price_scale = scaler_model.transform(test_price)


lr = LinearRegression(featuresCol = 'var_exp_scale', labelCol='price_per_person', maxIter=100, regParam = 0.1, elasticNetParam = 0)
lr_train_test = lr.fit(train_price_scale)

# on applique la prédiction
lr_predictions = lr_train_test.transform(test_price_scale)

# on sort MAE et rmse pour juger de la qualité de la prédiction
test_result = lr_train_test.evaluate(test_price_scale)
# RMSE & MAE
print(test_result.rootMeanSquaredError, test_result.meanAbsoluteError)

34.451784698523 19.541581700911077

La pénalisation n'a pas beaucoup d'impact. On peut tester d'autres modèles de régression pour voir si on arrive encore à baisser cette moyenne.

In [38]:
# Random Forest
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol="var_exp", labelCol='price_per_person')

rf_trained = rf.fit(train_price)
# on applique la prédiction
rf_predictions = rf_trained.transform(test_price)  

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price_per_person", predictionCol="prediction", metricName="mae")

print("Moyenne de l'erreur en valeur absolue (MAE) sur base de test = %g" % evaluator.evaluate(rf_predictions))


Moyenne de l'erreur en valeur absolue (MAE) sur base de test = 22.2648

Le modèle random forest fait moins bien que le modèle linéaire, essayons le gradient boosted trees

In [40]:
# Gradient Boosted Tree
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(featuresCol="var_exp", labelCol='price_per_person', maxIter=50)

gbt_trained = gbt.fit(train_price)
# on applique la prédiction
gbt_predictions = gbt_trained.transform(test_price)  

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price_per_person", predictionCol="prediction", metricName="mae")

print("Moyenne de l'erreur en valeur absolue (MAE) sur base de test = %g $" % evaluator.evaluate(gbt_predictions))


Moyenne de l'erreur en valeur absolue (MAE) sur base de test = 16.6441 $

C'est le gradient boosted tree qui nous donne les meilleurs résultats en termes de prédictions avec un MAE toujours inférieure aux précédents modèle et qui se situe autour de 16,6$.

## Modèle différencié par pays
Plutôt que de contrôler par `country_dummies`, on peut affiner et faire autant de modèles que de pays en mettant tout ça dans une fonction. Pour tenter d'améliorer un peu les prédictions et conserver une indicatrice géographique dans les modèles par pays, on peut ausi ajouter `market_dummies` qui correspond plus ou moins aux villes (maille géographique un peu plus large que la ville).

In [43]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

def pred_by_country(country, df = df):
  
  df = df.withColumn("reviews_per_month", df["reviews_per_month"].cast(FloatType()))
  df_num = df.filter(df.country == country).drop("country", "country_dummies")
  
  name_num = df_num.drop("price_per_person").columns
  vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
  # ! Attention, on a déjà une variable qui s'appelle features !
  df_reg = vectorAssembler.transform(df_num)
  
  # On splite
  splits = df_reg.randomSplit([0.7, 0.3])
  train_price = splits[0]
  test_price = splits[1]
  
  # On fait le GBT
  gbt = GBTRegressor(featuresCol="var_exp", labelCol='price_per_person', maxIter=50)

  gbt_trained = gbt.fit(train_price)
  # on applique la prédiction
  gbt_predictions = gbt_trained.transform(test_price)  

  # Select (prediction, true label) and compute test error
  evaluator = RegressionEvaluator(
  labelCol="price_per_person", predictionCol="prediction", metricName="mae")

  return(evaluator.evaluate(gbt_predictions))

Affichons les résultats de la prédiction pour certains pays :

In [45]:
res = pd.DataFrame(index=["United States", "United Kingdom", "France", "Spain", "Australia", "Italy", "Canada", "Germany", "Netherlands"],
                   columns=["MAE", "Prix moyen"])

# calcul des moyennes de pays
means = dict(df.groupBy("country").avg("price_per_person").collect())

In [46]:
# On découpe en 3 cellules sinon c'est trop long (plus de 2h...) et le cluster se termine
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["United States", "United Kingdom", "France"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [47]:
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["Spain", "Australia", "Italy"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [48]:
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["Canada", "Germany", "Netherlands"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [49]:
# ajout des erreurs relatives par pays  
res["Erreur absolue relative"] = res["MAE"]/res["Prix moyen"]
res

,MAE,Prix moyen,Erreur absolue relative
United States,18.0425,66.3385,0.271977
United Kingdom,10.2588,39.1737,0.26188
France,10.4693,41.0895,0.254794
Spain,8.8633,32.9147,0.269281
Australia,17.7789,68.8367,0.258277
Italy,8.76898,36.3542,0.24121
Canada,13.8952,52.4991,0.264676
Germany,8.50792,32.4495,0.26219
Netherlands,12.8696,61.2844,0.209998


Pour conclure, les performances du modèle varient un peu en fonction des pays avec une erreur absolue relative qui varie entre 21 et 27%.